## Using logprobs to ensure that an acrostic poem is generated

In [1]:
#%pip install --upgrade --quiet transformers torch fbgemm-gpu accelerate

In [2]:
# CHANGE this to the Llama model for which you have applied for access via Hugging Face
# See: https://www.llama.com/docs/getting-the-models/hugging-face/
MODEL_ID = "meta-llama/Llama-3.2-3B-Instruct"

In [3]:
import os
from dotenv import load_dotenv
load_dotenv("../keys.env")
assert os.environ["HF_TOKEN"][:2] == "hf",\
       "Please sign up for access to the specific Llama model via HuggingFace and provide access token in keys.env file"

## Zero-shot generation


In [4]:
from transformers import pipeline

pipe = pipeline(
    task="text-generation", 
    model=MODEL_ID,
    use_fast=True,
    kwargs={
        "return_full_text": False,
    },
    model_kwargs={}
)

/mnt/v/linuxHome/.virtualenvs/python3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 15.40it/s]
Device set to use cuda:0


In [5]:
def get_input_prompts(animal: str, poem_so_far=None, start_word=None):
    system_prompt="""
    You are a poet writing acrostic poems about animals for a childrens' book.
    The first letters of the acrostic poem need to spell out an adjective that is suitable for the animal.
    For example, the first letters of an acrostic poem about rabbits might spell out "quick" or "cute"
    and the entire poem needs to be a single phrase that describes the animal or something the animal
    might do.

    Thus, an acrostic poem about rabbits might be:

    Quietly, the rabbit bides its time
    Under the garden deck
    In wait for
    Carrot greens,
    Kale, and parsley.
    
    Respond with only the poem. Do not include any preamble or introduction.
    """
 
    if poem_so_far:
        user_prompt=f"""
        Complete the following acrostic poem about {animal}.
        The starting letters of the poem's lines need to spell out the word {start_word}.
        
        Poem:
        {poem_so_far}"""
    else:
        user_prompt=f"""
        Write an acrostic poem about {animal}.
        
        Poem:
        """
    
    return system_prompt, user_prompt
    

def get_input_message(animal: str, poem_so_far=None):
    system_prompt, user_prompt = get_input_prompts(animal, poem_so_far)
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

In [6]:
results = pipe(get_input_message("tiger"), max_new_tokens=256)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [7]:
print(results)

[{'generated_text': [{'role': 'system', 'content': '\n    You are a poet writing acrostic poems about animals for a childrens\' book.\n    The first letters of the acrostic poem need to spell out an adjective that is suitable for the animal.\n    For example, the first letters of an acrostic poem about rabbits might spell out "quick" or "cute"\n    and the entire poem needs to be a single phrase that describes the animal or something the animal\n    might do.\n\n    Thus, an acrostic poem about rabbits might be:\n\n    Quietly, the rabbit bides its time\n    Under the garden deck\n    In wait for\n    Carrot greens,\n    Kale, and parsley.\n\n    Respond with only the poem. Do not include any preamble or introduction.\n    '}, {'role': 'user', 'content': '\n        Write an acrostic poem about tiger.\n\n        Poem:\n        '}, {'role': 'assistant', 'content': 'Tenacious, the tiger stalks its prey\nEagerly, it pounces through the day\nRapidly, it runs with mighty pace\nIlluminating t

In [8]:
print(results[0]['generated_text'][-1]['content'])

Tenacious, the tiger stalks its prey
Eagerly, it pounces through the day
Rapidly, it runs with mighty pace
Illuminating the forest space
Gentle eyes, a secret hide


Our result:
```
Powerful eyes gleam in the night
Occupying shadows, a fierce sight
Ruling the forest with gentle might
Elegant, a creature of beauty bright
```
As you can see, the model probably tried to generate POWER as the starting letters, but failed.
There are several ways to fix this. Reflection might work, for example.
But in this notebook, we'll illustrate how to use logprobs to select the next best sequence that meets our style constraints.

## Getting logprobs of sequences, generating candidate starting words

In [9]:
inputs = pipe.tokenizer('\n'.join(get_input_prompts("tiger")), return_tensors="pt").to("cuda")

In [10]:
print(inputs)

{'input_ids': tensor([[128000,    198,    262,   1472,    527,    264,  40360,   4477,   1645,
          42602,    292,  45319,    922,  10099,    369,    264,   2911,     82,
              6,   2363,    627,    262,    578,   1176,  12197,    315,    279,
           1645,  42602,    292,  33894,   1205,    311,  13141,    704,    459,
          85592,    430,    374,  14791,    369,    279,  10065,    627,    262,
           1789,   3187,     11,    279,   1176,  12197,    315,    459,   1645,
          42602,    292,  33894,    922,  70244,   2643,  13141,    704,    330,
          28863,      1,    477,    330,     66,   1088,    702,    262,    323,
            279,   4553,  33894,   3966,    311,    387,    264,   3254,  17571,
            430,  16964,    279,  10065,    477,   2555,    279,  10065,    198,
            262,   2643,    656,    382,    262,  14636,     11,    459,   1645,
          42602,    292,  33894,    922,  70244,   2643,    387,   1473,    262,
          6858

In [11]:
# See: https://discuss.huggingface.co/t/announcement-generation-get-probabilities-for-generated-output/30075
results = pipe.model.generate(
    **inputs,
    max_new_tokens=16,
    num_beams=4,
    num_return_sequences=4,
    output_scores=True,
    renormalize_logits=True,
    return_dict_in_generate=True
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [12]:
print(results.sequences[0])

tensor([128000,    198,    262,   1472,    527,    264,  40360,   4477,   1645,
         42602,    292,  45319,    922,  10099,    369,    264,   2911,     82,
             6,   2363,    627,    262,    578,   1176,  12197,    315,    279,
          1645,  42602,    292,  33894,   1205,    311,  13141,    704,    459,
         85592,    430,    374,  14791,    369,    279,  10065,    627,    262,
          1789,   3187,     11,    279,   1176,  12197,    315,    459,   1645,
         42602,    292,  33894,    922,  70244,   2643,  13141,    704,    330,
         28863,      1,    477,    330,     66,   1088,    702,    262,    323,
           279,   4553,  33894,   3966,    311,    387,    264,   3254,  17571,
           430,  16964,    279,  10065,    477,   2555,    279,  10065,    198,
           262,   2643,    656,    382,    262,  14636,     11,    459,   1645,
         42602,    292,  33894,    922,  70244,   2643,    387,   1473,    262,
         68587,    398,     11,    279, 

In [13]:
print(results.sequences_scores)

tensor([-0.2643, -0.2685, -0.4117, -0.4663], device='cuda:0')


In [14]:
pipe.tokenizer.decode(results.sequences[0])

'<|begin_of_text|>\n    You are a poet writing acrostic poems about animals for a childrens\' book.\n    The first letters of the acrostic poem need to spell out an adjective that is suitable for the animal.\n    For example, the first letters of an acrostic poem about rabbits might spell out "quick" or "cute"\n    and the entire poem needs to be a single phrase that describes the animal or something the animal\n    might do.\n\n    Thus, an acrostic poem about rabbits might be:\n\n    Quietly, the rabbit bides its time\n    Under the garden deck\n    In wait for\n    Carrot greens,\n    Kale, and parsley.\n\n    Respond with only the poem. Do not include any preamble or introduction.\n    \n\n        Write an acrostic poem about tiger.\n\n        Poem:\n         T - Tenacious and strong\n         I - In the forest, it ro'

In [15]:
def get_poem_so_far(sequence: str):
    last_phrase = "Poem:"
    lines = sequence[sequence.find(last_phrase)+len(last_phrase):].split('\n')
    lines = [line.strip() for line in lines]
    lines = [line for line in lines if len(line) > 0] # non-empty lines
    #lines = [line for line in lines if len(line) > 2] # at least 3 letters
    return lines

candidate_poem = get_poem_so_far(pipe.tokenizer.decode(results.sequences[0]))
print(candidate_poem)

['T - Tenacious and strong', 'I - In the forest, it ro']


In [16]:
''.join([line[0] for line in candidate_poem]).lower()

'ti'

In [17]:
import re

def get_potential_starts(animal: str, num_words: int):
    system_prompt=f"""
    You are an expert on words who has access to a thesaurus.
    Respond with a list of adjectives that could complete the phrase "As ___ as a {animal}"
    For example, for a rabbit, you could respond with:   quick, fast, gentle, playful       
    Respond with just a list of words without any introduction or preamble.
    """
    user_prompt=f"Give me the best {num_words*3} adjectives that would complete the phrase 'As ___ as a {animal}'"
    input_message = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    result = pipe(input_message, max_new_tokens=256)
    words = result[0]['generated_text'][-1]['content']
    # remove numbering, commas, etc.
    words = re.compile('[^a-z]').sub(' ', words.lower()).split()
    # at least 4 letters
    words = [w for w in words if len(w) > 3]
    # get unique words
    words = list(set(words))
    # sort them by length
    words = sorted(words, key=lambda x: len(x))
    return words[:num_words]

start_words = get_potential_starts("tiger", 5)
print(start_words)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['agile', 'regal', 'savage', 'robust', 'fierce']


In [18]:
def get_phrase_that_starts_with(animal: str, letter: str):
    system_prompt=f"""
    You are writing a children's book. Write a phrase about a {animal} that starts with the letter {letter}
    Respond with just the phrase without an introduction or preamble.
    """
    user_prompt=f"Write a phrase about a {animal} that starts with the letter {letter}"
    input_message = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    result = pipe(input_message, max_new_tokens=256)
    phrase = result[0]['generated_text'][-1]['content']
    return ' '.join(phrase.split()[:3]) # max 3 words

get_phrase_that_starts_with("tiger", "N")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'Nimbly navigating the'

In [19]:
import numpy as np
import random

def initialize_poem(animal: str, allowed_start_words: [str]):
    # the weight of a word is inversely proportional to its length
    lengths = [1.0*len(w) for w in allowed_start_words]
    max_len = np.max(lengths)
    weights = (max_len - lengths)
    weights = weights / np.sum(weights)
    
    start_word = random.choices(population=allowed_start_words, weights=weights, k=1)[0].lower()
    start_letter = start_word[0].upper()
    return start_word, [get_phrase_that_starts_with(animal, start_letter)]

initialize_poem("tiger", start_words)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


('agile', ['Amidst the vibrant'])

## Putting it together

In [20]:
def make_unique(list_of_dict):
    unique_poems, unique_pd = set(), []
    # decreasing prob, so first one wins
    list_of_dict = sorted(list_of_dict, key=lambda x: x['prob'], reverse=True)
    for pd in list_of_dict:
        poem_str = '\n'.join(pd['poem'])
        if poem_str not in unique_poems:
            unique_poems.add(poem_str)
            unique_pd.append(pd)
    return unique_pd

def write_acrostic(animal: str, max_iter=10, num_sequences_per_iter=10, verbose_thresh=0):
    found_poems = []
    allowed_start_words = get_potential_starts(animal, 10)
    if verbose_thresh < 3:
        print(f"Allowed start words: {allowed_start_words}")
    
    start_word, poem_so_far, prev_start_poem = None, None, None
    for iter in range(max_iter):
        # reinitiatilize if we are stuck at a starting point
        if poem_so_far is None or poem_so_far == prev_start_poem:
            start_word, poem_so_far = initialize_poem(animal, allowed_start_words)  # lines of poem
        prev_start_poem = poem_so_far # for next iter
        if verbose_thresh < 2:
            print(f"Generating poem for {start_word} starting with {';'.join(poem_so_far)}")
            
        # generate poem
        inputs = pipe.tokenizer('\n'.join(get_input_prompts(animal, '\n'.join(poem_so_far), start_word)), 
                                return_tensors="pt").to("cuda")
        results = pipe.model.generate(
            **inputs,
            max_new_tokens=16,
            num_beams=num_sequences_per_iter,
            num_return_sequences=num_sequences_per_iter,
            output_scores=True,
            renormalize_logits=True,
            return_dict_in_generate=True,
        )
        
        # choose tokens based on logprobs and whether it meets style
        best_prob_in_iter = -1000 # remember these are logits
        best_poem_in_iter = None
        best_bad_prob_in_iter = -1000
        best_bad_poem_in_iter = None
        for seqno, sequence in enumerate(results.sequences):
            seq_prob = results.sequences_scores[seqno]
            candidate_poem = get_poem_so_far(pipe.tokenizer.decode(sequence))
            if len(candidate_poem) > len(start_word):
                candidate_poem = candidate_poem[:len(start_word)] # truncate poem to length of start word
            if verbose_thresh < 1:
                print(f"Iter: {iter} Word: {start_word} Seqno: {seqno} Prob: {seq_prob}  Poem: {';'.join(candidate_poem)}")
            # is this in the list of possibilities?
            candidate_starts = ''.join([line[0] for line in candidate_poem]).lower()
            continue_seq = False
            found_poem = False
            if len(start_word) >= len(candidate_starts) and start_word[:len(candidate_starts)] == candidate_starts:
                continue_seq = True
                if len(start_word) == len(candidate_starts):
                    found_poems.append({
                        "poem": candidate_poem, 
                        "prob": float(np.exp(seq_prob.cpu())),
                        "word": start_word
                    }) # YEAH!
                    if verbose_thresh < 3:
                        print(f"Found poem: {found_poems[-1]}")               
                
            if continue_seq:
                if seq_prob > best_prob_in_iter:
                    best_prob_in_iter = seq_prob
                    # even if a poem is found, the last line might be incomplete, so continue sequence
                    best_poem_in_iter = candidate_poem
            else:
                if seq_prob > best_bad_prob_in_iter:
                    best_bad_prob_in_iter = seq_prob
                    best_bad_poem_in_iter = candidate_poem
        
        # update the starting point for the next iteration
        if best_poem_in_iter:
            poem_so_far = best_poem_in_iter
        else:
            # remove the lines that don't fit and try again
            print(f"REMOVING from {';'.join(best_bad_poem_in_iter)}")
            while True:
                # remove a line, and see if it matches the start word
                best_bad_poem_in_iter = best_bad_poem_in_iter[:-1]
                if len(best_bad_poem_in_iter) == 0:
                    # reinitialize, potentially to different start word
                    start_word, poem_so_far = initialize_poem(animal, allowed_start_words)
                    break
                candidate_starts = ''.join([line[0] for line in best_bad_poem_in_iter]).lower()
                if len(start_word) >= len(candidate_starts) and start_word[:len(candidate_starts)] == candidate_starts:
                    poem_so_far = best_bad_poem_in_iter # start from here, for same start_word
                    break            
    
    return make_unique(found_poems)
    
found_poems = write_acrostic("tiger", 10)           

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Allowed start words: ['wild', 'bold', 'agile', 'regal', 'swift', 'fierce', 'brutal', 'strong', 'savage', 'potent']


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Generating poem for bold starting with Brightly colored stripes


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Generating poem for bold starting with Boldly, the brave


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Iter: 7 Word: bold Seqno: 0 Prob: -0.1874387413263321  Poem: Boldly, the brave tiger stalks its prey;Owning the forest with its might
Iter: 7 Word: bold Seqno: 1 Prob: -0.25473955273628235  Poem: Boldly, the brave tiger stalks its prey;Owning the forest, night and day
Iter: 7 Word: bold Seqno: 2 Prob: -0.2686450183391571  Poem: Boldly, the brave tiger roams;Owning the forest's darkest night;Lur
Iter: 7 Word: bold Seqno: 3 Prob: -0.2739299535751343  Poem: Boldly, the brave tiger stalks;Owning the forest floor;Lurking in
Iter: 7 Word: bold Seqno: 4 Prob: -0.28247883915901184  Poem: Boldly, the brave tiger stalks its prey;Owning the jungle with its might
Iter: 7 Word: bold Seqno: 5 Prob: -0.2968197464942932  Poem: Boldly, the brave tiger stalks;Owning the forest deep;Lurking in
Iter: 7 Word: bold Seqno: 6 Prob: -0.2976932227611542  Poem: Boldly, the brave tiger roams;Owning the forest deep;Lurking in
Iter: 7 Word: bold Seqno: 7 Prob: -0.30612003803253174  Poem: Boldly, the brave tiger sta

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Iter: 8 Word: bold Seqno: 0 Prob: -0.2220558524131775  Poem: Boldly, the brave tiger stalks its prey;Owning the forest with its might;Lurking in the shadows, waiting to pounce;Daring to
Found poem: {'poem': ['Boldly, the brave tiger stalks its prey', 'Owning the forest with its might', 'Lurking in the shadows, waiting to pounce', 'Daring to'], 'prob': 0.8008705973625183, 'word': 'bold'}
Iter: 8 Word: bold Seqno: 1 Prob: -0.26967689394950867  Poem: Boldly, the brave tiger stalks its prey;Owning the forest with its might;Lurking in the shadows, silent as can be;Daring to
Found poem: {'poem': ['Boldly, the brave tiger stalks its prey', 'Owning the forest with its might', 'Lurking in the shadows, silent as can be', 'Daring to'], 'prob': 0.763626217842102, 'word': 'bold'}
Iter: 8 Word: bold Seqno: 2 Prob: -0.27624988555908203  Poem: Boldly, the brave tiger stalks its prey;Owning the forest with its might;Lurking in the shadows, it waits;Daring to pounce
Found poem: {'poem': ['Boldly, the br

Sometimes, the generation fails.  For example, with the start word of ferocious, the algo generates:
```
Fiercely fearless, Freddy the tiger roams
Eagerly exploring every forest home
Rapidly racing through the trees
Oozing orange and black with ease
```
but then none of the generated sequences start with the letter C.  We could make the algorithm backtrack and restart at the third line ...

In [21]:
found_poems

[{'poem': ['Brightly colored stripes shine',
   'Owning the forest floor',
   'Lively eyes watch all',
   'Daring to roam free'],
  'prob': 0.8781275153160095,
  'word': 'bold'},
 {'poem': ['Boldly, the brave tiger stalks its prey',
   'Owning the forest with its might',
   'Lurking in the shadows, waiting to pounce',
   'Daring to be the king of the night'],
  'prob': 0.8159196376800537,
  'word': 'bold'},
 {'poem': ['Boldly, the brave tiger stalks its prey',
   'Owning the forest with its might',
   'Lurking in the shadows, waiting to pounce',
   'Daring to'],
  'prob': 0.8008705973625183,
  'word': 'bold'},
 {'poem': ['Boldly, the brave tiger stalks its prey',
   'Owning the forest with its might',
   'Lurking in the shadows, silent as can be',
   'Daring to'],
  'prob': 0.763626217842102,
  'word': 'bold'},
 {'poem': ['Boldly, the brave tiger stalks its prey',
   'Owning the forest with its might',
   'Lurking in the shadows, it waits',
   'Daring to pounce'],
  'prob': 0.758623301

Some results:

(1) BOLD:
```
Boldly, the brave tiger stalks its prey
Owning the forest with its might,
Lurking in the shadows, waiting to pounce,
Daring to be the king of the night
```

(2) AGILE:
```
Among the tall trees, a tiger resides
Gently, it stalks its prey
Icy winds blow through the forest
Lurking, it waits for the perfect moment
Eager to pounce and catch its dinner
```

(3) SWIFT:
```
Smiling softly, the
Wild eyes gleam
In the
Forest depths,
Tigers stalk
```

Sure, not the greatest poems, but illustrative of how you can generate in segments and use logprobs to select the segment that meets style constraints.

In [22]:
# let's do a different animal
write_acrostic("owl", max_iter=10, verbose_thresh=2)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Allowed start words: ['keen', 'calm', 'wise', 'still', 'sharp', 'alert', 'deadly', 'fierce', 'silent', 'patient']


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


REMOVING from Softly swooping through the night;Silent wings, a gentle flight;In the shadows,


...

Found poem: {'poem': ['With wise eyes, the owl surveys', 'In the dark of night', 'Silent as a ghost', 'Eyes shining bright'], 'prob': 0.7006473541259766, 'word': 'wise'}
Found poem: {'poem': ['With wise eyes, the owl surveys', 'In the dark of night', 'Silent as a ghost', 'Ears perked up high'], 'prob': 0.6902604103088379, 'word': 'wise'}
Found poem: {'poem': ['With wise eyes, the owl surveys', 'In the dark of night', 'Silent as a ghost', 'Ears perked up high'], 'prob': 0.6672167778015137, 'word': 'wise'}
Found poem: {'poem': ['With wise eyes, the owl surveys', 'In the dark of night', 'Silent as a ghost', 'Ears perked up high'], 'prob': 0.6588158011436462, 'word': 'wise'}
Found poem: {'poem': ['With wise eyes, the owl surveys', 'In the dark of night', 'Silent as a ghost', 'Ears perked up high'], 'prob': 0.6579573154449463, 'word': 'wise'}
Found poem: {'poem': ['With wise eyes, the owl surveys', 'In the dark of night', 'Silent as a ghost', 'Ears perked up high'], 'prob': 0.64935916662216

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Found poem: {'poem': ['With wise eyes, the owl surveys', 'In the dark of night', 'Silent as a ghost', 'Eyes shining bright'], 'prob': 0.7858545184135437, 'word': 'wise'}
Found poem: {'poem': ['With wise eyes, the owl surveys', 'In the dark of night', 'Silent as a ghost', 'Eyes shining bright'], 'prob': 0.7667717337608337, 'word': 'wise'}
Found poem: {'poem': ['With wise eyes, the owl surveys', 'In the dark of night', 'Silent as a ghost', 'Eyes shining bright'], 'prob': 0.7331704497337341, 'word': 'wise'}
Found poem: {'poem': ['With wise eyes, the owl surveys', 'In the dark of night', 'Silent as a ghost', 'Eyes shining bright'], 'prob': 0.7280787825584412, 'word': 'wise'}
Found poem: {'poem': ['With wise eyes, the owl surveys', 'In the dark of night', 'Silent as a ghost', 'Eyes shining bright'], 'prob': 0.7277273535728455, 'word': 'wise'}
Found poem: {'poem': ['With wise eyes, the owl surveys', 'In the dark of night', 'Silent as a ghost', 'Eyes shining bright'], 'prob': 0.72469502687454

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'poem': ['With wise eyes, the owl surveys',
   'In the dark of night',
   'Silent as a ghost',
   'Eyes shining bright'],
  'prob': 0.7858545184135437,
  'word': 'wise'},
 {'poem': ['Amidst the moonlit night',
   'Lurks a silent hunter',
   'Ears perked up high',
   'Rays of moonlight dance',
   'Talons outstretched, ready'],
  'prob': 0.7707433104515076,
  'word': 'alert'},
 {'poem': ['Amidst the moonlit night',
   'Lurks a silent hunter',
   'Ears perked up high',
   'Rays of moonlight dance',
   'Talons outstretched, ready to strike'],
  'prob': 0.7340291738510132,
  'word': 'alert'},
 {'poem': ['With wise eyes, the owl surveys',
   'In the dark of night',
   'Silent as a ghost',
   'Ears perked up high'],
  'prob': 0.6902604103088379,
  'word': 'wise'},
 {'poem': ['Amidst the moonlit night',
   'Lurks a silent hunter',
   'Ears perked up high',
   'Rays of moonlight dance',
   'Talons outstretched,'],
  'prob': 0.6660639047622681,
  'word': 'alert'},
 {'poem': ['Amidst the moonli

Results:
```
Among the moonlit trees
Gently, the owl takes flight
Icy winds do not deter
Lurking shadows hide its sight
Elegant wings, a silent glide
```
and
```
Silently swooping through the night
Hooting softly, a gentle sound
Alertly watching, with eyes so bright
Ruling the darkness, all around
Prowling quietly, with stealthy pace
```
and
```
Quietly, the owl's wings unfold
Under the moon's silver glow
In the darkness, it takes flight
Eyes shining bright with wisdom's light
Talons sharp, it soars with ease
```